# Imports

In [1]:
import numpy as np
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import collections
from sklearn.preprocessing import MinMaxScaler
from random import randint

import pickle

print(torch.__version__)
import time

import os
import shutil
import copy

!pip install mido
import mido

1.8.1+cu101
     |████████████████████████████████| 61kB 3.3MB/s 


# Mido Helper Functions

In [2]:
from mido import Message, MidiFile, MidiTrack
import copy

def listToMidi(chord_list,
               dir_path = './',
               file_name = "new_song.mid",
               program = 12,
               dt = 256,
               veloON = 64,
               veloOFF = 127,
               printOut = True,
               saveMidiFile=True,
               completePath = False):
  """
  converts midi value chords into a midi file with given specifications.
  """
  mid = MidiFile()
  track = MidiTrack()
  mid.tracks.append(track)

  #initialize instrument type
  track.append(Message('program_change', program=12, time=0))
  #Loop through each all chords in list
  for chord in chord_list:
    start = True
    
    #turn chord(s) on
    for note in chord:
      if start == True:
        
        track.append(Message('note_on', note=note, velocity=veloON, time=dt))
        start = False
      else:
        track.append(Message('note_on', note=note, velocity=veloON, time=0))

    start = True
    #turn chord(s) off
    for note in chord:
      if start == True:
        track.append(Message('note_off', note=note, velocity=veloOFF, time=dt))
        start = False
      else:
        track.append(Message('note_off', note=note, velocity=veloOFF, time=0))

  #print created track
  if printOut == True:
    for msg in mid:
      print(msg)
  
  #save midi file
  if saveMidiFile == True and completePath == False:
    complete_path = os.path.join(dir_path,file_name)
    #print(complete_path)
    mid.save(complete_path)
    print("\nfile saved @\n{}".format(complete_path))
  elif completePath != False:
    mid.save(completePath)
##########################################################
##########################################################
#NOT WORKING RIGHT NOW
def midiExtend(file_path,dupNum=2,save_path = False):
  mid = MidiFile(file_path)

  final_tracks = MidiTrack()
  for i in range(dupNum):
    for track in mid:
      #print(track)
      final_tracks.append(track)

  #create new file name
  file_name = "midi_extended"
  num = str(dupNum)
  extension = ".mid"
  full_name = file_name + num + extension

  path = "/".join(file_path.split("/")[0:-1])
  #print("Path {}\n".format(path))
  #print(full_name)
  mid_final = MidiFile()
  mid_final.tracks.append(final_tracks)
  
  final_path = os.path.join(path,full_name)
  mid_final.save(final_path)

# Load Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tensorLocation = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Datasets/AllDataSetsCombined/Essential_Midi_ProgressionsOnly/tensor_essentialDataset.pt"
dataset = torch.load(tensorLocation)

print(dataset.shape)

(1416, 4, 3)


## Embedding

In [5]:
def datasetEmbedder(dataset,inputLength=3):
  """
  dataset: tensorflow dataset with dimensions (total_progs,progression length,notes in chord)
  inputLength: # of chords played.

  Function seperates progressions into "input" & "target"
  """
  assert len(dataset[0]) > inputLength, "inputLength needs to be less than progression length"
  #quadgrams = [(progression[:3],progression[3]) for progression in dataset]
  if inputLength != len(dataset[0]):
    grams = [(progression[:inputLength],progression[inputLength]) for progression in dataset]
  
  return grams

def decoder(code=0):
  """
  code: input encrypted recommendation from model
  """
  string = ix_to_chord[code]
  string = string.replace("["," ")
  string = string.replace("]"," ")
  string = list(string.split(" "))
  final_chord = [int(x) for x in string if x != ""]
  return final_chord

#######################Lookup Table Functions###################################
def createLookup(dataset):
  #create chords as a singular word
  vocab = []
  for prog in dataset:
    for chord in prog:
      tmp_chord = str(chord.numpy()) #convert chord to string
      vocab.append(tmp_chord)
  vocab = set(vocab)
  print("# of Chords in Vocab: {}".format(len(vocab)))

  #create lookup table
  word_to_ix = {word:i for i,word in enumerate(vocab)} #encode
  ix_to_chord = {i:word for i,word in enumerate(vocab)} #decode

  return vocab, word_to_ix, ix_to_chord

def saveLookUp(encode_dict,decode_dict,dirPath):
  """
  encode: word_to_ix
  decode: ix_to_chord
  """
  file_encode = r"lookUp_encode.txt"
  file_decode = r"lookUp_decode.txt"
  #save files
  with open(os.path.join(dirPath,file_encode), "wb") as myFile_encode:
    pickle.dump(encode_dict, myFile_encode)

  with open(os.path.join(dirPath,file_decode), "wb") as myFile_decode:
    pickle.dump(decode_dict, myFile_decode)

  print("Files Saved @:\n{}".format(dirPath))

def loadLookUp(dirPath):
  """
  reverse of saveLookUp()
  """
  file_encode = r"lookUp_encode.txt"
  file_decode = r"lookUp_decode.txt"

  with open(os.path.join(dirPath,file_encode), "rb") as myFile_encode:
    encode_dict = pickle.load(myFile_encode)

  with open(os.path.join(dirPath,file_decode), "rb") as myFile_decode:
    decode_dict = pickle.load(myFile_decode)

  assert len(encode_dict) == len(decode_dict), "encode and decode must be same length"

  vocab = encode_dict #used for the len(vocab) for embed dimensions

  return encode_dict, decode_dict, vocab
########################################################################
########################################################################

Start of Initialization(s)

In [6]:
#####################EMBEDDING CREATION####################
CONTEXT_SIZE = 3 #decides how long the input is for next recommendation
EMBEDDING_DIM = 100 #how many dimensions we want inside the linear model to have

###seperates progressions into input and target as a tuple###
quadgrams = datasetEmbedder(dataset,inputLength=CONTEXT_SIZE)

###create new LookUp Table from Dataset###
#vocab, word_to_ix, ix_to_chord = createLookup(dataset) #create new lookup table

#save Lookup Table(s)
saveDir = r"/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/lookup_tables"
#saveLookUp(word_to_ix,ix_to_chord,saveDir)

###load Lookup Table(s)###
word_to_ix, ix_to_chord,vocab = loadLookUp(saveDir)


print("Encoder\n{}".format(word_to_ix))
print("Decoder\n{}".format(ix_to_chord))
print("Number of Chord Options: {}".format(len(word_to_ix)))

Encoder
{'[53 56 60]': 0, '[46 49 53]': 1, '[45 48 52]': 2, '[47 50 53]': 3, '[54 57 61]': 4, '[58 62 65]': 5, '[61 64 67]': 6, '[70 73 76]': 7, '[68 71 74]': 8, '[65 68 71]': 9, '[57 61 64]': 10, '[59 63 66]': 11, '[61 65 68]': 12, '[50 54 57]': 13, '[45 49 52]': 14, '[53 57 60]': 15, '[47 50 54]': 16, '[55 58 62]': 17, '[61 64 68]': 18, '[51 54 57]': 19, '[60 63 67]': 20, '[59 62 66]': 21, '[48 52 55]': 22, '[64 67 70]': 23, '[66 70 73]': 24, '[54 58 61]': 25, '[49 52 55]': 26, '[67 70 74]': 27, '[43 46 50]': 28, '[57 60 64]': 29, '[54 57 60]': 30, '[65 68 72]': 31, '[48 51 55]': 32, '[62 66 69]': 33, '[63 67 70]': 34, '[66 69 73]': 35, '[60 63 66]': 36, '[50 53 56]': 37, '[41 44 48]': 38, '[64 67 71]': 39, '[64 68 71]': 40, '[68 72 75]': 41, '[62 65 69]': 42, '[58 61 65]': 43, '[44 47 51]': 44, '[52 55 59]': 45, '[48 51 54]': 46, '[42 45 49]': 47, '[67 70 73]': 48, '[66 69 72]': 49, '[53 56 59]': 50, '[49 53 56]': 51, '[49 52 56]': 52, '[68 71 75]': 53, '[55 58 61]': 54, '[51 54 58]

In [7]:
#EXAMPLE: Shows that x list is numbers that pertain to each chord in context
for context, target in quadgrams:
  x = [word_to_ix[str(w.numpy())] for w in context]

print(quadgrams[-1][1].numpy())
print(x)

[58 62 65]
[17, 15, 17]


# Model

In [8]:
#helping function --> Max Output

def modelRecommend(modelOutput, output_random = False,random_len = 2):
  """
  Used to convert model_output to midi note values
  """
  ##Extract Data from OUTPUT##
  modelOut = modelOutput.cpu()
  modelOut = modelOut.detach().numpy()
  modelOut = list(modelOut)
  finalOut = list(modelOut[0])
  ############################

  #Method Decision --> Max vs Random
  if output_random == False:
    recommendedChord = finalOut.index(max(finalOut))
  else:
    #create copy with values sorted
    out_sorted = copy.copy(finalOut)
    out_sorted.sort(reverse = True) #Order greatest to least
    out_sorted = out_sorted[:random_len]
    ###############################################################
    #grab indexes of greatest values based on desired input length
    top_choice_indexes = []
    for i in range(len(out_sorted)):
      top_choice_indexes.append(finalOut.index(out_sorted[i]))
    ###############################################################
    #random selector
    recommendedChord = randint(0,len(top_choice_indexes)-1)
  
  return recommendedChord

def modelRecommend_max(modelOutput):
  recommended_max = modelRecommend(modelOutput)
  return recommended_max

def testModelOut(modelOutput):
  modelOut = modelOutput.cpu()
  modelOut = modelOut.detach().numpy()
  modelOut = list(modelOut)
  finalOut = list(modelOut[0])

  out_sorted = copy.copy(finalOut)
  out_sorted.sort(reverse = True) #Order greatest to least

  top_choice_indexes = []
  for i in range(len(out_sorted)):
    top_choice_indexes.append(finalOut.index(out_sorted[i]))

  print(top_choice_indexes)

In [9]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

Start of Training Model

In [10]:
#initalize GPU & model
gpu = torch.cuda.get_device_name(0)

#intialize
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training

In [11]:
#train
for epoch in range(100):
    total_loss = 0

    total_accuracy = 0
    count = 0
    correct = 0

    flag = True #change to false for testing
    for context, target in quadgrams:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in context], dtype=torch.long).cuda()

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model.forward(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[str(target.numpy())]], dtype=torch.long).cuda())

        #debug code
        if epoch == 0 and flag == False:
          print("Length of Model Output: ", len(log_probs[0]))
          print("model output: {}\nTarget Chord: {}".format(log_probs,
                                            word_to_ix[str(target.numpy())]))
          flag = True


        #Step 4a. Calculate Accuracy
        recommendedChord = modelRecommend(log_probs)
        count += 1
        if recommendedChord == word_to_ix[str(target.numpy())]:
          correct += 1

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()

    total_accuracy = correct/count
    print("Epoch: {}  Loss: {}  Accuracy: {}".format(epoch,total_loss,total_accuracy))
    losses.append(total_loss)

#final results of training session
print("Loss: ",losses)

torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])
torch.Size([1, 77])


KeyboardInterrupt: ignored

## Save and Load Weights into Primary model

In [ ]:
#save model weights - MAKE SURE TO NOT OVERWRITE
model_weights_save_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/embed_state_dict_model_example.pt"
torch.save(model.state_dict(),model_weights_save_path)
print("weights saved")

weights saved


In [ ]:
#LOAD MODEL WEIGHTS
load_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/embed_state_dict_model_1inputs_100_dims.pt"
model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

# Evaluation

In [ ]:
#evaluation of model
total_loss = 0

total_accuracy = 0
counter = 0
correct = 0

flag = True #change to false for testing
model.eval()
with torch.no_grad():
  for context, target in quadgrams:
      # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
      # into integer indices and wrap them in tensors)
      context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in context], dtype=torch.long).cuda()

      # Step 3. Run the forward pass, getting log probabilities over next
      # words
      log_probs = model.forward(context_idxs)

      # Step 4. Compute your loss function. (Again, Torch wants the target
      # word wrapped in a tensor)
      loss = loss_function(log_probs, torch.tensor([word_to_ix[str(target.numpy())]], dtype=torch.long).cuda())


      #Step 4a. Calculate Accuracy
      recommendedChord = modelRecommend(log_probs)
      counter += 1
      expected = word_to_ix[str(target.numpy())]
      if recommendedChord == expected:
        correct += 1
        #print("Count: {}\nCORRECT\nrecommendedChord: {}\nExpected: {}\n".format(counter,recommendedChord,expected))
      else:
        pass
        #print("Count: {}\nFAIL\nrecommendedChord: {}\nExpected: {}\n".format(counter,recommendedChord,expected))

      # Get the Python number from a 1-element Tensor by calling tensor.item()
      total_loss += loss.item()

  total_accuracy = correct/counter
  print("Loss: {}  Accuracy: {}".format(total_loss,total_accuracy))

#Inference Test
print("\n\nInference Speed Test...")
t_start = time.time()
test_data = quadgrams[0]
input = test_data[0]
target = test_data[1]

model.eval()
with torch.no_grad():
  context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in input], dtype=torch.long).cuda()

  log_probs = model(context_idxs)
  ##########################################
  #prints index order based on greatest to least values
  testModelOut(log_probs)
  ###########################################
  recommendedChord = modelRecommend(log_probs)
  final_chord = decoder(recommendedChord)
  print("final chord: {} dict_number: {}".format(final_chord,recommendedChord))
  print("Eval Time: {}(seconds)".format(time.time() - t_start))

Loss: 1098.7358681841922  Accuracy: 0.58545197740113


Inference Speed Test...
[39, 75, 33, 62, 29, 21, 56, 22, 45, 49, 0, 35, 60, 25, 40, 16, 17, 54, 55, 58, 76, 42, 67, 30, 26, 38, 36, 18, 3, 46, 47, 15, 73, 28, 31, 44, 72, 1, 5, 37, 9, 66, 65, 61, 53, 2, 51, 19, 27, 57, 14, 48, 52, 12, 6, 71, 63, 41, 70, 74, 32, 64, 10, 69, 7, 59, 50, 23, 34, 68, 8, 43, 13, 11, 20, 24, 4]
final chord: [64, 67, 71] dict_number: 39
Eval Time: 0.003604888916015625(seconds)


# Results
                Eval Accuracy w/ 100 epochs  dimension sizes of (10,100)
1. 3 chord inputs: (.55911,.58811)  
2. 2 Chord inputs: (.474576,.4845)  
3. 1 Chord inputs: (.235875,.2295) 

## Inference function and run code

Inference Loading and Run Functions

In [ ]:
### intialize multiple models for 1,2, and 3 inputs
EMBEDDING_DIM = 100 #how many dimensions we want inside the linear model to have

#1-INPUT model
CONTEXT_SIZE = 1
model1 = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).cuda()

load_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/embed_state_dict_model_1inputs_100_dims.pt"
model1.load_state_dict(torch.load(load_path))

#2-INPUT model
CONTEXT_SIZE = 2
model2 = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).cuda()

load_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/embed_state_dict_model_2inputs_100_dims.pt"
model2.load_state_dict(torch.load(load_path))

#3-INPUT model
CONTEXT_SIZE = 3
model3 = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).cuda()

load_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/embed_state_dict_model_3inputs_100_dims.pt"
model3.load_state_dict(torch.load(load_path))
print("All models Loaded Successfully\n")

#intialize dictionary for function call uses
model_dict = {1: model1,2:model2,3:model3}
print(model_dict)

All models Loaded Successfully

{1: NGramLanguageModeler(
  (embeddings): Embedding(77, 100)
  (linear1): Linear(in_features=100, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=77, bias=True)
), 2: NGramLanguageModeler(
  (embeddings): Embedding(77, 100)
  (linear1): Linear(in_features=200, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=77, bias=True)
), 3: NGramLanguageModeler(
  (embeddings): Embedding(77, 100)
  (linear1): Linear(in_features=300, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=77, bias=True)
)}


In [ ]:
#model run functions
def modelRunMultiple(TESTDATA,saveMidi=True):
  modelRun(TESTDATA,saveMidi=saveMidi)

def modelRun(TESTDATA,saveMidi=True):
  #inference for chunk of dataset
  model.eval()
  with torch.no_grad():
    count = 0
    midi_out = []
    for context, target in TESTDATA:
      #time grabber for inference calculations
      dt = time.time()
      #convert input to model form
      context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in context], dtype=torch.long).cuda()
      #print(context)

      #run model
      log_probs = model(context_idxs)
      #print("model output: {}\n".format(log_probs))

      #Chord Selection
      recommendedChord = modelRecommend(log_probs)
      #print("encoded recomendation {}\n".format(recommendedChord))

      #Decode cord
      recommendation = decoder(code=recommendedChord)
      #print("recommendation: {}\n".format(recommendation))

      #append decode to input for musicCompletor
      midi_input = np.append(context,[recommendation],axis=0)
      #print(midi_input)

      #save to midi file
      file_name = BASE_FILE_NAME + str(count) + HANDLE
      file_path = os.path.join(SAVE_DIR,file_name)
      #print("path\n{}".format(file_path))
      count += 1
      if saveMidi == True:
        listToMidi(midi_input,completePath=file_path,printOut=False)
        print("file saved at:\n{}".format(file_path))
        
      print("Song #: {}\nInference Time: {}".format(count,time.time()-dt))

      midi_out.append(midi_input)
  return midi_out

########################################################################################
#####################SELF CREATING CODE FOR 4 CHORD PROGRESSION OF 1,2, & 3 INPUTS######
def modelRunSingle(dataIn,model,saveMidi=True,output_random=False):
  #inference for chunk of dataset
  model.eval()
  with torch.no_grad():
    #seperate tuple of input data
    context = dataIn

    count = 0
    midi_out = []
    #time grabber for inference calculations
    dt = time.time()
    #convert input to model form
    context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in context], dtype=torch.long).cuda()
    #print(context)

    #run model
    log_probs = model(context_idxs)
    #print("model output: {}\n".format(log_probs))

    #Chord Selection
    recommendedChord = modelRecommend(log_probs,output_random=output_random)#########################################################################################
    #print("encoded recomendation {}\n".format(recommendedChord))

    #Decode cord
    recommendation = decoder(code=recommendedChord)
    #print("recommendation: {}\n".format(recommendation))

    #append decode to input for musicCompletor
    midi_input = np.append(context,[recommendation],axis=0)
    #print(midi_input)

    #save to midi file
    file_name = BASE_FILE_NAME + str(count) + HANDLE
    file_path = os.path.join(SAVE_DIR,file_name)
    #print("path\n{}".format(file_path))
    count += 1
    if saveMidi == True:
      listToMidi(midi_input,completePath=file_path,printOut=False)
      print("file saved at:\n{}".format(file_path))

    midi_out.append(midi_input)
    return midi_out

def chordCreator_SingleInput(inputData,output_random = False):
  #initalize starting length, chord progression cant go past 4 chords in given state
  inputLength = len(inputData[0])
  CONTEXT_SIZE = inputLength
  #print("Initial Progression Length: {}".format(inputLength))
  assert type(inputLength) == type(1), "inputLength must be Integer"

  assert inputLength < 4, "chord progression DOES not fit. Please make sure progression is between 1 and 3"
  model = model_dict[inputLength]

  #loop through each chord decision to create multiple chords at once
  #inference form only uses modelRunSingle with Recursive code
  firstPass = True
  progLength = inputLength #changes per iteration  but starts with starting progression length
  input, _ = inputData
  while progLength < 4:
    #first iteration
    if firstPass == True:
      newProg = modelRunSingle(input,model,saveMidi=False,output_random=output_random)
      progLength = len(newProg[0])
      #print("Post-FirstPass progLength: {}".format(progLength))
      firstPass = False
    else:
      newProg = tf.squeeze(tf.convert_to_tensor(newProg)) #convert to model form
      model = model_dict[progLength]

      newProg = modelRunSingle(newProg,model,saveMidi=False,output_random=output_random)
      progLength = len(newProg[0])
      #print("progLength: ", progLength)
  return newProg[0]

def multipleProgs(sub_quadgrams,fileNameStart = "test_",saveFiles = True, filterCopies=False,output_random=False):
  """
  use with selection of quadgrams/inputs of "quadgram" form to save multiple midi chord progressions made by model
  """
  progs = []
  for i, prog in enumerate(sub_quadgrams):
    single_progression = chordCreator_SingleInput(prog,output_random=output_random)
    #print("single progression Info\nType: {}\nData: {}".format(type(single_progression),single_progression))
    file_name = fileNameStart + str(i) + ".mid"
    if saveFiles == True:
      listToMidi(single_progression,file_name=file_name,printOut=False)
    progs.append(single_progression)

  if filterCopies == True:
    progs = copyFilter(progs)
  
  return progs

def copyFilter(list_of_recommendations,printChords = False):
  print("\n\nNumber of Chord Progressions: {}".format(len(list_of_recommendations)))
  res = []
  for x in list_of_recommendations:
    if res != []:
      is_found = []
      for y in res:
        #print(x==y)
        #print((x==y).all())
        if (x==y).all() == True:
          is_found.append("True")
        else:
          is_found.append("False")
      copy_found = "True" not in is_found
      if copy_found == True:
        res.append(x)
      
    else:
      res.append(x)

  print("Number of Unique Chord Progressions: {}".format(len(res)))

  if printChords == True:
    print("Original Chord Recs")
    for chord in list_of_recommendations:
      print(chord)
    print("Filtered Chord Progressions")
    for chord in res:
      print(chord)
      

  return res

##################################END OF FUNCTIONS FOR MUSIC GENERATION########################

---  
Single Chord Recommendation

In [ ]:
#variables
TESTDATA = list(quadgrams[50:60])
SAVE_DIR = r"/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/midiOutputs"
BASE_FILE_NAME = r"song_"
HANDLE = r".mid"

In [ ]:
#input must be one set of data from dataset

assert type(TESTDATA) == type([]), "Make sure model input is of type list"

prog = chordCreator_SingleInput(TESTDATA)
print("Created Progression:\n{}".format(prog))

Initial Progression Length: 1
Post-FirstPass progLength: 2
progLength:  3
progLength:  4
Created Progression:
[[55 59 62]
 [62 66 69]
 [64 67 71]
 [57 60 64]]


---  
Multi Chord Progression Creator

In [ ]:
TESTDATA = list(quadgrams[0:10])
out = multipleProgs(TESTDATA,fileNameStart="test500_start_",saveFiles=True,filterCopies=True,output_random=True)

for i,prog in enumerate(out):
  listToMidi(prog,file_name=BASE_FILE_NAME + str(i) + HANDLE,printOut=False)
print("Midi Conversion Complete")

#print(out)
#TO DO#
#set up saves with "out" variable, and disable "save=True" in inference functions

###DEBUG###
#remove saving from "filterCopies portion"


file saved @
./test500_start_0.mid

file saved @
./test500_start_1.mid

file saved @
./test500_start_2.mid

file saved @
./test500_start_3.mid

file saved @
./test500_start_4.mid

file saved @
./test500_start_5.mid

file saved @
./test500_start_6.mid

file saved @
./test500_start_7.mid

file saved @
./test500_start_8.mid

file saved @
./test500_start_9.mid


Number of Chord Progressions: 10
Number of Unique Chord Progressions: 8

file saved @
./song_0.mid

file saved @
./song_1.mid

file saved @
./song_2.mid

file saved @
./song_3.mid

file saved @
./song_4.mid

file saved @
./song_5.mid

file saved @
./song_6.mid

file saved @
./song_7.mid
Midi Conversion Complete


In [ ]:
tmp = out[0]

print(type(tmp[0][0]))

<class 'numpy.int64'>


## Load a Midi File and Play it in Collab

In [ ]:
#################GOOGLE COLLAB ONLY####################
#use these imports to play midi files in google collab#
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2 #copy file to direct location for synth
!pip install midi2audio
from midi2audio import FluidSynth
from IPython.display import Audio

#file path for midi saves
#/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/midiOutputs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 30 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni

In [ ]:
#CHANGE AS NEEDED
dir_path = r"/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/midiOutputs"
file_name = "new_song.mid"
output_name = "test.wav"

PATH = os.path.join(dir_path,file_name)
outputPATH = os.path.join(dir_path,output_name)

FluidSynth("font.sf2").midi_to_audio(PATH, outputPATH)
Audio(outputPATH)